<a href="https://colab.research.google.com/github/NoCodeProgram/deepLearning/blob/main/rnn/charRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/NoCodeProgram/deepLearning.git

Cloning into 'deepLearning'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 166 (delta 10), reused 0 (delta 0), pack-reused 139
Receiving objects: 100% (166/166), 4.38 MiB | 11.41 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [3]:
data_file = './deepLearning/rnn/name_country.csv'

In [4]:
import pandas as pd
import torch

df = pd.read_csv(data_file)
text_data = df['Name'].tolist()
label_data = df['Country'].tolist()

country_list = sorted(set(label_data))
country_count = len(country_list)

data_dict = {} #key-country, value - list of names
for name, country in zip(text_data, label_data):
    if country not in data_dict:
        data_dict[country] = []
    data_dict[country].append(name)

# print(data_dict)

In [5]:
all_letters = "abcdefghijklmnopqrstuvwxyz ,'"
n_letters = len(all_letters)

def nameToTensor(name):
    tensor = torch.zeros(len(name), n_letters)
    for char_idx, char in enumerate(name):
        letter_idx = all_letters.find(char)
        assert letter_idx != -1, f"char is {name}, {char}"
        tensor[char_idx][letter_idx] = 1
    return tensor


In [6]:
import torch.nn as nn
#Try other arch
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()

        self.hidden_size = hidden_size
        self.hidden = torch.zeros(1, self.hidden_size)

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)


    def forward(self, input):
        combined = torch.cat((input, self.hidden), 1)
        self.hidden = torch.tanh(self.i2h(combined))
        output = self.i2o(combined)
        return output

    def init_hidden(self):
        self.hidden = torch.zeros(1, self.hidden_size)


n_hidden = 32
rnn_model = RNN(n_letters, n_hidden, country_count)

In [ ]:
import random
from torch.optim import Adam, SGD
loss_fn = nn.CrossEntropyLoss()

optimizer = Adam(rnn_model.parameters(), lr=0.0001)
iter_count = 3000000
crnt_loss = 0.
correct_predictions = 0

for iter_idx in range(iter_count):
    rnn_model.train()
    random_country = random.choice(list(data_dict.keys()))
    random_name = random.choice(data_dict[random_country])

    name_tensor = nameToTensor(random_name)
    country_tensor = torch.tensor([country_list.index(random_country)], dtype=torch.long)
    rnn_model.init_hidden()
    rnn_model.zero_grad()

    for char_idx in range(len(random_name)):
        char_tensor = name_tensor[char_idx]
        output = rnn_model(char_tensor[None,:])

    loss = loss_fn(output, country_tensor)
    loss.backward()
    optimizer.step()

    crnt_loss += loss.item()
    predicted_index = torch.argmax(output, 1)
    correct_predictions += (predicted_index == country_tensor).sum().item()

    if iter_idx % 10000 == 0 and iter_idx != 0:
        average_loss = crnt_loss / 10000
        accuracy = 100 * correct_predictions / 10000
        print(f'Iter idx {iter_idx}, Loss: {average_loss:.4f}, Accuracy: {accuracy:.2f}%')
        crnt_loss = 0
        correct_predictions = 0
        total_predictions = 0

Iter idx 5000, Loss: 2.8493, Accuracy: 12.74%
Iter idx 10000, Loss: 2.6855, Accuracy: 25.56%
Iter idx 15000, Loss: 2.4757, Accuracy: 26.50%
Iter idx 20000, Loss: 2.3423, Accuracy: 32.56%
Iter idx 25000, Loss: 2.2325, Accuracy: 35.84%
Iter idx 30000, Loss: 2.1403, Accuracy: 37.86%
Iter idx 35000, Loss: 2.0694, Accuracy: 38.50%
Iter idx 40000, Loss: 2.0094, Accuracy: 40.30%
Iter idx 45000, Loss: 1.9383, Accuracy: 42.54%
Iter idx 50000, Loss: 1.8923, Accuracy: 43.74%
Iter idx 55000, Loss: 1.8705, Accuracy: 44.18%
Iter idx 60000, Loss: 1.8038, Accuracy: 45.70%
Iter idx 65000, Loss: 1.7718, Accuracy: 45.56%
Iter idx 70000, Loss: 1.7530, Accuracy: 45.58%
Iter idx 75000, Loss: 1.6834, Accuracy: 48.44%
Iter idx 80000, Loss: 1.6826, Accuracy: 47.84%
Iter idx 85000, Loss: 1.6501, Accuracy: 48.96%
Iter idx 90000, Loss: 1.6232, Accuracy: 48.20%
Iter idx 95000, Loss: 1.6086, Accuracy: 49.32%
Iter idx 100000, Loss: 1.5948, Accuracy: 49.76%
Iter idx 105000, Loss: 1.5772, Accuracy: 50.66%
Iter idx 110

In [ ]:
test_name = 'nocope'
test_name_tensor = nameToTensor(test_name)

rnn_model.eval()
rnn_model.init_hidden()
for char_idx in range(len(test_name)):
    char_tensor = test_name_tensor[char_idx]
    output = rnn_model(char_tensor[None,:])
predicted_index = torch.argmax(output, 1).item()
print(country_list[predicted_index])
